In [1]:
import numpy as np

In [184]:
np.set_printoptions(edgeitems=10,linewidth=390)
scan.shape

(25, 12)

In [221]:
# inputs = '''498,4 -> 498,6 -> 496,6
# 503,4 -> 502,4 -> 502,9 -> 494,9'''.split('\n')


paths = [np.array([tuple(map(int, tok.split(','))) for tok in path_str.split(' -> ')]) for path_str in inputs]
start_loc = np.array((500,0))

corners = np.concatenate(paths)
xmin, xmax = corners[:,0].min(), corners[:,0].max()
ymin, ymax = corners[:,1].min(), corners[:,1].max()


# Part 2
ymax = ymax + 2
xmin = min(xmin, start_loc[0] - (ymax + 1))
xmax = max(xmax, start_loc[0] + (ymax + 1))

axis_mins = np.array([xmin, 0])
start_loc = start_loc - axis_mins

# paths.append(np.array(((xmin,ymax), (xmax, ymax))))

scan = np.zeros((xmax - xmin + 1, ymax + 1), np.int32)
scan[:,-1] = 1

tok_to_idx = lambda x: x - axis_mins

def parse_path(path_toks, scan):
    for i, start in enumerate(path_toks[:-1]):
        end = path_toks[i+1]
        if start[0] == end[0]: axis = 1
        elif start[1] == end[1]: axis = 0
        else: raise ValueError(start, end)
        # print(i, start[axis], end[axis])
        for j in np.linspace(start[axis], end[axis], abs(start[axis] - end[axis]) + 1):
            tok = start + 0 # copy
            tok[axis] = j
            scan[tuple(tok_to_idx(tok))] = 1
            # print(i, j)
            # print(tok, start, end)

# parse_path(paths[0], scan)
# parse_path(paths[1], scan)
for path in paths:
    parse_path(path, scan)
    


In [222]:
from numba import jit, njit

@njit
def in_bounds(loc, scan):
    return all(loc >= 0) and all(loc < np.array(scan.shape))

# @jit
def _particle_loc_1step(loc, scan):
    move = np.array([0,1])
    for xmove in [0, -1, 1]:
        move[0] = xmove
        new_loc = loc + move
        if not in_bounds(new_loc, scan) or scan[tuple(new_loc)] == 0: return new_loc
    return loc

# @njit    
def get_fall_loc(start_loc, scan):
    while True:
        next_loc = _particle_loc_1step(start_loc, scan)
        if not in_bounds(next_loc, scan) or all(next_loc == start_loc): return next_loc
#         scan2 = scan + 0
#         scan2[tuple(next_loc)] = 3
#         print(scan2.T)        
        start_loc = next_loc

# def simulate_particles(start_loc, scan):
n_steps = 0
while scan[tuple(start_loc)] == 0:
    fall_loc = get_fall_loc(start_loc, scan)
    # print('here')
    if not in_bounds(fall_loc, scan): break
    scan[tuple(fall_loc)] = 2
    # print(scan.T)
    # print('\n\n')
    n_steps += 1

n_steps

30762

In [218]:
scan.T

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 1, 2, 2, 2, 1, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 2, 2, 1, 1, 1, 2, 2, 2, 1, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 2, 2, 2, 2, 0, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0],
       [0, 0, 0, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0],
       [0, 0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [110]:
n_steps

23

In [219]:
inputs = '''521,171 -> 525,171
497,147 -> 497,137 -> 497,147 -> 499,147 -> 499,146 -> 499,147 -> 501,147 -> 501,142 -> 501,147 -> 503,147 -> 503,144 -> 503,147 -> 505,147 -> 505,140 -> 505,147
465,76 -> 465,71 -> 465,76 -> 467,76 -> 467,72 -> 467,76 -> 469,76 -> 469,68 -> 469,76 -> 471,76 -> 471,73 -> 471,76 -> 473,76 -> 473,71 -> 473,76 -> 475,76 -> 475,70 -> 475,76 -> 477,76 -> 477,71 -> 477,76 -> 479,76 -> 479,73 -> 479,76 -> 481,76 -> 481,74 -> 481,76 -> 483,76 -> 483,72 -> 483,76
476,63 -> 476,58 -> 476,63 -> 478,63 -> 478,55 -> 478,63 -> 480,63 -> 480,56 -> 480,63 -> 482,63 -> 482,57 -> 482,63 -> 484,63 -> 484,55 -> 484,63 -> 486,63 -> 486,55 -> 486,63
503,109 -> 507,109
465,76 -> 465,71 -> 465,76 -> 467,76 -> 467,72 -> 467,76 -> 469,76 -> 469,68 -> 469,76 -> 471,76 -> 471,73 -> 471,76 -> 473,76 -> 473,71 -> 473,76 -> 475,76 -> 475,70 -> 475,76 -> 477,76 -> 477,71 -> 477,76 -> 479,76 -> 479,73 -> 479,76 -> 481,76 -> 481,74 -> 481,76 -> 483,76 -> 483,72 -> 483,76
503,150 -> 503,154 -> 501,154 -> 501,162 -> 513,162 -> 513,154 -> 507,154 -> 507,150
488,106 -> 488,97 -> 488,106 -> 490,106 -> 490,100 -> 490,106 -> 492,106 -> 492,101 -> 492,106 -> 494,106 -> 494,105 -> 494,106 -> 496,106 -> 496,96 -> 496,106 -> 498,106 -> 498,102 -> 498,106 -> 500,106 -> 500,101 -> 500,106 -> 502,106 -> 502,102 -> 502,106 -> 504,106 -> 504,100 -> 504,106
503,150 -> 503,154 -> 501,154 -> 501,162 -> 513,162 -> 513,154 -> 507,154 -> 507,150
481,79 -> 481,81 -> 478,81 -> 478,89 -> 490,89 -> 490,81 -> 485,81 -> 485,79
488,38 -> 493,38
465,76 -> 465,71 -> 465,76 -> 467,76 -> 467,72 -> 467,76 -> 469,76 -> 469,68 -> 469,76 -> 471,76 -> 471,73 -> 471,76 -> 473,76 -> 473,71 -> 473,76 -> 475,76 -> 475,70 -> 475,76 -> 477,76 -> 477,71 -> 477,76 -> 479,76 -> 479,73 -> 479,76 -> 481,76 -> 481,74 -> 481,76 -> 483,76 -> 483,72 -> 483,76
476,63 -> 476,58 -> 476,63 -> 478,63 -> 478,55 -> 478,63 -> 480,63 -> 480,56 -> 480,63 -> 482,63 -> 482,57 -> 482,63 -> 484,63 -> 484,55 -> 484,63 -> 486,63 -> 486,55 -> 486,63
482,93 -> 496,93 -> 496,92
476,63 -> 476,58 -> 476,63 -> 478,63 -> 478,55 -> 478,63 -> 480,63 -> 480,56 -> 480,63 -> 482,63 -> 482,57 -> 482,63 -> 484,63 -> 484,55 -> 484,63 -> 486,63 -> 486,55 -> 486,63
497,29 -> 502,29
465,76 -> 465,71 -> 465,76 -> 467,76 -> 467,72 -> 467,76 -> 469,76 -> 469,68 -> 469,76 -> 471,76 -> 471,73 -> 471,76 -> 473,76 -> 473,71 -> 473,76 -> 475,76 -> 475,70 -> 475,76 -> 477,76 -> 477,71 -> 477,76 -> 479,76 -> 479,73 -> 479,76 -> 481,76 -> 481,74 -> 481,76 -> 483,76 -> 483,72 -> 483,76
465,76 -> 465,71 -> 465,76 -> 467,76 -> 467,72 -> 467,76 -> 469,76 -> 469,68 -> 469,76 -> 471,76 -> 471,73 -> 471,76 -> 473,76 -> 473,71 -> 473,76 -> 475,76 -> 475,70 -> 475,76 -> 477,76 -> 477,71 -> 477,76 -> 479,76 -> 479,73 -> 479,76 -> 481,76 -> 481,74 -> 481,76 -> 483,76 -> 483,72 -> 483,76
497,147 -> 497,137 -> 497,147 -> 499,147 -> 499,146 -> 499,147 -> 501,147 -> 501,142 -> 501,147 -> 503,147 -> 503,144 -> 503,147 -> 505,147 -> 505,140 -> 505,147
488,106 -> 488,97 -> 488,106 -> 490,106 -> 490,100 -> 490,106 -> 492,106 -> 492,101 -> 492,106 -> 494,106 -> 494,105 -> 494,106 -> 496,106 -> 496,96 -> 496,106 -> 498,106 -> 498,102 -> 498,106 -> 500,106 -> 500,101 -> 500,106 -> 502,106 -> 502,102 -> 502,106 -> 504,106 -> 504,100 -> 504,106
488,106 -> 488,97 -> 488,106 -> 490,106 -> 490,100 -> 490,106 -> 492,106 -> 492,101 -> 492,106 -> 494,106 -> 494,105 -> 494,106 -> 496,106 -> 496,96 -> 496,106 -> 498,106 -> 498,102 -> 498,106 -> 500,106 -> 500,101 -> 500,106 -> 502,106 -> 502,102 -> 502,106 -> 504,106 -> 504,100 -> 504,106
465,76 -> 465,71 -> 465,76 -> 467,76 -> 467,72 -> 467,76 -> 469,76 -> 469,68 -> 469,76 -> 471,76 -> 471,73 -> 471,76 -> 473,76 -> 473,71 -> 473,76 -> 475,76 -> 475,70 -> 475,76 -> 477,76 -> 477,71 -> 477,76 -> 479,76 -> 479,73 -> 479,76 -> 481,76 -> 481,74 -> 481,76 -> 483,76 -> 483,72 -> 483,76
465,76 -> 465,71 -> 465,76 -> 467,76 -> 467,72 -> 467,76 -> 469,76 -> 469,68 -> 469,76 -> 471,76 -> 471,73 -> 471,76 -> 473,76 -> 473,71 -> 473,76 -> 475,76 -> 475,70 -> 475,76 -> 477,76 -> 477,71 -> 477,76 -> 479,76 -> 479,73 -> 479,76 -> 481,76 -> 481,74 -> 481,76 -> 483,76 -> 483,72 -> 483,76
501,32 -> 506,32
488,106 -> 488,97 -> 488,106 -> 490,106 -> 490,100 -> 490,106 -> 492,106 -> 492,101 -> 492,106 -> 494,106 -> 494,105 -> 494,106 -> 496,106 -> 496,96 -> 496,106 -> 498,106 -> 498,102 -> 498,106 -> 500,106 -> 500,101 -> 500,106 -> 502,106 -> 502,102 -> 502,106 -> 504,106 -> 504,100 -> 504,106
490,16 -> 490,20 -> 484,20 -> 484,26 -> 499,26 -> 499,20 -> 494,20 -> 494,16
503,115 -> 507,115
524,173 -> 528,173
481,79 -> 481,81 -> 478,81 -> 478,89 -> 490,89 -> 490,81 -> 485,81 -> 485,79
486,47 -> 490,47
497,147 -> 497,137 -> 497,147 -> 499,147 -> 499,146 -> 499,147 -> 501,147 -> 501,142 -> 501,147 -> 503,147 -> 503,144 -> 503,147 -> 505,147 -> 505,140 -> 505,147
476,63 -> 476,58 -> 476,63 -> 478,63 -> 478,55 -> 478,63 -> 480,63 -> 480,56 -> 480,63 -> 482,63 -> 482,57 -> 482,63 -> 484,63 -> 484,55 -> 484,63 -> 486,63 -> 486,55 -> 486,63
490,16 -> 490,20 -> 484,20 -> 484,26 -> 499,26 -> 499,20 -> 494,20 -> 494,16
476,63 -> 476,58 -> 476,63 -> 478,63 -> 478,55 -> 478,63 -> 480,63 -> 480,56 -> 480,63 -> 482,63 -> 482,57 -> 482,63 -> 484,63 -> 484,55 -> 484,63 -> 486,63 -> 486,55 -> 486,63
495,50 -> 499,50
492,41 -> 496,41
476,63 -> 476,58 -> 476,63 -> 478,63 -> 478,55 -> 478,63 -> 480,63 -> 480,56 -> 480,63 -> 482,63 -> 482,57 -> 482,63 -> 484,63 -> 484,55 -> 484,63 -> 486,63 -> 486,55 -> 486,63
465,76 -> 465,71 -> 465,76 -> 467,76 -> 467,72 -> 467,76 -> 469,76 -> 469,68 -> 469,76 -> 471,76 -> 471,73 -> 471,76 -> 473,76 -> 473,71 -> 473,76 -> 475,76 -> 475,70 -> 475,76 -> 477,76 -> 477,71 -> 477,76 -> 479,76 -> 479,73 -> 479,76 -> 481,76 -> 481,74 -> 481,76 -> 483,76 -> 483,72 -> 483,76
497,147 -> 497,137 -> 497,147 -> 499,147 -> 499,146 -> 499,147 -> 501,147 -> 501,142 -> 501,147 -> 503,147 -> 503,144 -> 503,147 -> 505,147 -> 505,140 -> 505,147
522,177 -> 522,178 -> 536,178 -> 536,177
465,76 -> 465,71 -> 465,76 -> 467,76 -> 467,72 -> 467,76 -> 469,76 -> 469,68 -> 469,76 -> 471,76 -> 471,73 -> 471,76 -> 473,76 -> 473,71 -> 473,76 -> 475,76 -> 475,70 -> 475,76 -> 477,76 -> 477,71 -> 477,76 -> 479,76 -> 479,73 -> 479,76 -> 481,76 -> 481,74 -> 481,76 -> 483,76 -> 483,72 -> 483,76
488,106 -> 488,97 -> 488,106 -> 490,106 -> 490,100 -> 490,106 -> 492,106 -> 492,101 -> 492,106 -> 494,106 -> 494,105 -> 494,106 -> 496,106 -> 496,96 -> 496,106 -> 498,106 -> 498,102 -> 498,106 -> 500,106 -> 500,101 -> 500,106 -> 502,106 -> 502,102 -> 502,106 -> 504,106 -> 504,100 -> 504,106
503,121 -> 507,121
503,150 -> 503,154 -> 501,154 -> 501,162 -> 513,162 -> 513,154 -> 507,154 -> 507,150
515,121 -> 519,121
488,106 -> 488,97 -> 488,106 -> 490,106 -> 490,100 -> 490,106 -> 492,106 -> 492,101 -> 492,106 -> 494,106 -> 494,105 -> 494,106 -> 496,106 -> 496,96 -> 496,106 -> 498,106 -> 498,102 -> 498,106 -> 500,106 -> 500,101 -> 500,106 -> 502,106 -> 502,102 -> 502,106 -> 504,106 -> 504,100 -> 504,106
476,63 -> 476,58 -> 476,63 -> 478,63 -> 478,55 -> 478,63 -> 480,63 -> 480,56 -> 480,63 -> 482,63 -> 482,57 -> 482,63 -> 484,63 -> 484,55 -> 484,63 -> 486,63 -> 486,55 -> 486,63
498,35 -> 503,35
465,76 -> 465,71 -> 465,76 -> 467,76 -> 467,72 -> 467,76 -> 469,76 -> 469,68 -> 469,76 -> 471,76 -> 471,73 -> 471,76 -> 473,76 -> 473,71 -> 473,76 -> 475,76 -> 475,70 -> 475,76 -> 477,76 -> 477,71 -> 477,76 -> 479,76 -> 479,73 -> 479,76 -> 481,76 -> 481,74 -> 481,76 -> 483,76 -> 483,72 -> 483,76
503,171 -> 507,171
509,121 -> 513,121
494,12 -> 494,13 -> 507,13 -> 507,12
465,76 -> 465,71 -> 465,76 -> 467,76 -> 467,72 -> 467,76 -> 469,76 -> 469,68 -> 469,76 -> 471,76 -> 471,73 -> 471,76 -> 473,76 -> 473,71 -> 473,76 -> 475,76 -> 475,70 -> 475,76 -> 477,76 -> 477,71 -> 477,76 -> 479,76 -> 479,73 -> 479,76 -> 481,76 -> 481,74 -> 481,76 -> 483,76 -> 483,72 -> 483,76
500,112 -> 504,112
488,124 -> 488,126 -> 485,126 -> 485,129 -> 496,129 -> 496,126 -> 491,126 -> 491,124
476,63 -> 476,58 -> 476,63 -> 478,63 -> 478,55 -> 478,63 -> 480,63 -> 480,56 -> 480,63 -> 482,63 -> 482,57 -> 482,63 -> 484,63 -> 484,55 -> 484,63 -> 486,63 -> 486,55 -> 486,63
488,124 -> 488,126 -> 485,126 -> 485,129 -> 496,129 -> 496,126 -> 491,126 -> 491,124
476,63 -> 476,58 -> 476,63 -> 478,63 -> 478,55 -> 478,63 -> 480,63 -> 480,56 -> 480,63 -> 482,63 -> 482,57 -> 482,63 -> 484,63 -> 484,55 -> 484,63 -> 486,63 -> 486,55 -> 486,63
512,173 -> 516,173
465,76 -> 465,71 -> 465,76 -> 467,76 -> 467,72 -> 467,76 -> 469,76 -> 469,68 -> 469,76 -> 471,76 -> 471,73 -> 471,76 -> 473,76 -> 473,71 -> 473,76 -> 475,76 -> 475,70 -> 475,76 -> 477,76 -> 477,71 -> 477,76 -> 479,76 -> 479,73 -> 479,76 -> 481,76 -> 481,74 -> 481,76 -> 483,76 -> 483,72 -> 483,76
502,38 -> 507,38
488,106 -> 488,97 -> 488,106 -> 490,106 -> 490,100 -> 490,106 -> 492,106 -> 492,101 -> 492,106 -> 494,106 -> 494,105 -> 494,106 -> 496,106 -> 496,96 -> 496,106 -> 498,106 -> 498,102 -> 498,106 -> 500,106 -> 500,101 -> 500,106 -> 502,106 -> 502,102 -> 502,106 -> 504,106 -> 504,100 -> 504,106
465,76 -> 465,71 -> 465,76 -> 467,76 -> 467,72 -> 467,76 -> 469,76 -> 469,68 -> 469,76 -> 471,76 -> 471,73 -> 471,76 -> 473,76 -> 473,71 -> 473,76 -> 475,76 -> 475,70 -> 475,76 -> 477,76 -> 477,71 -> 477,76 -> 479,76 -> 479,73 -> 479,76 -> 481,76 -> 481,74 -> 481,76 -> 483,76 -> 483,72 -> 483,76
503,150 -> 503,154 -> 501,154 -> 501,162 -> 513,162 -> 513,154 -> 507,154 -> 507,150
497,147 -> 497,137 -> 497,147 -> 499,147 -> 499,146 -> 499,147 -> 501,147 -> 501,142 -> 501,147 -> 503,147 -> 503,144 -> 503,147 -> 505,147 -> 505,140 -> 505,147
497,147 -> 497,137 -> 497,147 -> 499,147 -> 499,146 -> 499,147 -> 501,147 -> 501,142 -> 501,147 -> 503,147 -> 503,144 -> 503,147 -> 505,147 -> 505,140 -> 505,147
506,169 -> 510,169
490,16 -> 490,20 -> 484,20 -> 484,26 -> 499,26 -> 499,20 -> 494,20 -> 494,16
497,147 -> 497,137 -> 497,147 -> 499,147 -> 499,146 -> 499,147 -> 501,147 -> 501,142 -> 501,147 -> 503,147 -> 503,144 -> 503,147 -> 505,147 -> 505,140 -> 505,147
503,150 -> 503,154 -> 501,154 -> 501,162 -> 513,162 -> 513,154 -> 507,154 -> 507,150
465,76 -> 465,71 -> 465,76 -> 467,76 -> 467,72 -> 467,76 -> 469,76 -> 469,68 -> 469,76 -> 471,76 -> 471,73 -> 471,76 -> 473,76 -> 473,71 -> 473,76 -> 475,76 -> 475,70 -> 475,76 -> 477,76 -> 477,71 -> 477,76 -> 479,76 -> 479,73 -> 479,76 -> 481,76 -> 481,74 -> 481,76 -> 483,76 -> 483,72 -> 483,76
465,76 -> 465,71 -> 465,76 -> 467,76 -> 467,72 -> 467,76 -> 469,76 -> 469,68 -> 469,76 -> 471,76 -> 471,73 -> 471,76 -> 473,76 -> 473,71 -> 473,76 -> 475,76 -> 475,70 -> 475,76 -> 477,76 -> 477,71 -> 477,76 -> 479,76 -> 479,73 -> 479,76 -> 481,76 -> 481,74 -> 481,76 -> 483,76 -> 483,72 -> 483,76
488,124 -> 488,126 -> 485,126 -> 485,129 -> 496,129 -> 496,126 -> 491,126 -> 491,124
491,121 -> 495,121
497,147 -> 497,137 -> 497,147 -> 499,147 -> 499,146 -> 499,147 -> 501,147 -> 501,142 -> 501,147 -> 503,147 -> 503,144 -> 503,147 -> 505,147 -> 505,140 -> 505,147
518,173 -> 522,173
501,50 -> 505,50
497,115 -> 501,115
498,47 -> 502,47
476,63 -> 476,58 -> 476,63 -> 478,63 -> 478,55 -> 478,63 -> 480,63 -> 480,56 -> 480,63 -> 482,63 -> 482,57 -> 482,63 -> 484,63 -> 484,55 -> 484,63 -> 486,63 -> 486,55 -> 486,63
488,106 -> 488,97 -> 488,106 -> 490,106 -> 490,100 -> 490,106 -> 492,106 -> 492,101 -> 492,106 -> 494,106 -> 494,105 -> 494,106 -> 496,106 -> 496,96 -> 496,106 -> 498,106 -> 498,102 -> 498,106 -> 500,106 -> 500,101 -> 500,106 -> 502,106 -> 502,102 -> 502,106 -> 504,106 -> 504,100 -> 504,106
488,106 -> 488,97 -> 488,106 -> 490,106 -> 490,100 -> 490,106 -> 492,106 -> 492,101 -> 492,106 -> 494,106 -> 494,105 -> 494,106 -> 496,106 -> 496,96 -> 496,106 -> 498,106 -> 498,102 -> 498,106 -> 500,106 -> 500,101 -> 500,106 -> 502,106 -> 502,102 -> 502,106 -> 504,106 -> 504,100 -> 504,106
465,76 -> 465,71 -> 465,76 -> 467,76 -> 467,72 -> 467,76 -> 469,76 -> 469,68 -> 469,76 -> 471,76 -> 471,73 -> 471,76 -> 473,76 -> 473,71 -> 473,76 -> 475,76 -> 475,70 -> 475,76 -> 477,76 -> 477,71 -> 477,76 -> 479,76 -> 479,73 -> 479,76 -> 481,76 -> 481,74 -> 481,76 -> 483,76 -> 483,72 -> 483,76
518,169 -> 522,169
476,63 -> 476,58 -> 476,63 -> 478,63 -> 478,55 -> 478,63 -> 480,63 -> 480,56 -> 480,63 -> 482,63 -> 482,57 -> 482,63 -> 484,63 -> 484,55 -> 484,63 -> 486,63 -> 486,55 -> 486,63
489,44 -> 493,44
465,76 -> 465,71 -> 465,76 -> 467,76 -> 467,72 -> 467,76 -> 469,76 -> 469,68 -> 469,76 -> 471,76 -> 471,73 -> 471,76 -> 473,76 -> 473,71 -> 473,76 -> 475,76 -> 475,70 -> 475,76 -> 477,76 -> 477,71 -> 477,76 -> 479,76 -> 479,73 -> 479,76 -> 481,76 -> 481,74 -> 481,76 -> 483,76 -> 483,72 -> 483,76
481,79 -> 481,81 -> 478,81 -> 478,89 -> 490,89 -> 490,81 -> 485,81 -> 485,79
509,167 -> 513,167
476,63 -> 476,58 -> 476,63 -> 478,63 -> 478,55 -> 478,63 -> 480,63 -> 480,56 -> 480,63 -> 482,63 -> 482,57 -> 482,63 -> 484,63 -> 484,55 -> 484,63 -> 486,63 -> 486,55 -> 486,63
487,133 -> 487,134 -> 501,134
481,79 -> 481,81 -> 478,81 -> 478,89 -> 490,89 -> 490,81 -> 485,81 -> 485,79
488,106 -> 488,97 -> 488,106 -> 490,106 -> 490,100 -> 490,106 -> 492,106 -> 492,101 -> 492,106 -> 494,106 -> 494,105 -> 494,106 -> 496,106 -> 496,96 -> 496,106 -> 498,106 -> 498,102 -> 498,106 -> 500,106 -> 500,101 -> 500,106 -> 502,106 -> 502,102 -> 502,106 -> 504,106 -> 504,100 -> 504,106
488,106 -> 488,97 -> 488,106 -> 490,106 -> 490,100 -> 490,106 -> 492,106 -> 492,101 -> 492,106 -> 494,106 -> 494,105 -> 494,106 -> 496,106 -> 496,96 -> 496,106 -> 498,106 -> 498,102 -> 498,106 -> 500,106 -> 500,101 -> 500,106 -> 502,106 -> 502,102 -> 502,106 -> 504,106 -> 504,100 -> 504,106
512,118 -> 516,118
490,16 -> 490,20 -> 484,20 -> 484,26 -> 499,26 -> 499,20 -> 494,20 -> 494,16
488,106 -> 488,97 -> 488,106 -> 490,106 -> 490,100 -> 490,106 -> 492,106 -> 492,101 -> 492,106 -> 494,106 -> 494,105 -> 494,106 -> 496,106 -> 496,96 -> 496,106 -> 498,106 -> 498,102 -> 498,106 -> 500,106 -> 500,101 -> 500,106 -> 502,106 -> 502,102 -> 502,106 -> 504,106 -> 504,100 -> 504,106
512,165 -> 516,165
465,76 -> 465,71 -> 465,76 -> 467,76 -> 467,72 -> 467,76 -> 469,76 -> 469,68 -> 469,76 -> 471,76 -> 471,73 -> 471,76 -> 473,76 -> 473,71 -> 473,76 -> 475,76 -> 475,70 -> 475,76 -> 477,76 -> 477,71 -> 477,76 -> 479,76 -> 479,73 -> 479,76 -> 481,76 -> 481,74 -> 481,76 -> 483,76 -> 483,72 -> 483,76
465,76 -> 465,71 -> 465,76 -> 467,76 -> 467,72 -> 467,76 -> 469,76 -> 469,68 -> 469,76 -> 471,76 -> 471,73 -> 471,76 -> 473,76 -> 473,71 -> 473,76 -> 475,76 -> 475,70 -> 475,76 -> 477,76 -> 477,71 -> 477,76 -> 479,76 -> 479,73 -> 479,76 -> 481,76 -> 481,74 -> 481,76 -> 483,76 -> 483,72 -> 483,76
491,35 -> 496,35
500,173 -> 504,173
488,106 -> 488,97 -> 488,106 -> 490,106 -> 490,100 -> 490,106 -> 492,106 -> 492,101 -> 492,106 -> 494,106 -> 494,105 -> 494,106 -> 496,106 -> 496,96 -> 496,106 -> 498,106 -> 498,102 -> 498,106 -> 500,106 -> 500,101 -> 500,106 -> 502,106 -> 502,102 -> 502,106 -> 504,106 -> 504,100 -> 504,106
490,16 -> 490,20 -> 484,20 -> 484,26 -> 499,26 -> 499,20 -> 494,20 -> 494,16
503,150 -> 503,154 -> 501,154 -> 501,162 -> 513,162 -> 513,154 -> 507,154 -> 507,150
483,50 -> 487,50
481,79 -> 481,81 -> 478,81 -> 478,89 -> 490,89 -> 490,81 -> 485,81 -> 485,79
500,118 -> 504,118
497,147 -> 497,137 -> 497,147 -> 499,147 -> 499,146 -> 499,147 -> 501,147 -> 501,142 -> 501,147 -> 503,147 -> 503,144 -> 503,147 -> 505,147 -> 505,140 -> 505,147
488,106 -> 488,97 -> 488,106 -> 490,106 -> 490,100 -> 490,106 -> 492,106 -> 492,101 -> 492,106 -> 494,106 -> 494,105 -> 494,106 -> 496,106 -> 496,96 -> 496,106 -> 498,106 -> 498,102 -> 498,106 -> 500,106 -> 500,101 -> 500,106 -> 502,106 -> 502,102 -> 502,106 -> 504,106 -> 504,100 -> 504,106
488,106 -> 488,97 -> 488,106 -> 490,106 -> 490,100 -> 490,106 -> 492,106 -> 492,101 -> 492,106 -> 494,106 -> 494,105 -> 494,106 -> 496,106 -> 496,96 -> 496,106 -> 498,106 -> 498,102 -> 498,106 -> 500,106 -> 500,101 -> 500,106 -> 502,106 -> 502,102 -> 502,106 -> 504,106 -> 504,100 -> 504,106
492,47 -> 496,47
488,106 -> 488,97 -> 488,106 -> 490,106 -> 490,100 -> 490,106 -> 492,106 -> 492,101 -> 492,106 -> 494,106 -> 494,105 -> 494,106 -> 496,106 -> 496,96 -> 496,106 -> 498,106 -> 498,102 -> 498,106 -> 500,106 -> 500,101 -> 500,106 -> 502,106 -> 502,102 -> 502,106 -> 504,106 -> 504,100 -> 504,106
497,147 -> 497,137 -> 497,147 -> 499,147 -> 499,146 -> 499,147 -> 501,147 -> 501,142 -> 501,147 -> 503,147 -> 503,144 -> 503,147 -> 505,147 -> 505,140 -> 505,147
476,63 -> 476,58 -> 476,63 -> 478,63 -> 478,55 -> 478,63 -> 480,63 -> 480,56 -> 480,63 -> 482,63 -> 482,57 -> 482,63 -> 484,63 -> 484,55 -> 484,63 -> 486,63 -> 486,55 -> 486,63
495,38 -> 500,38
494,12 -> 494,13 -> 507,13 -> 507,12
476,63 -> 476,58 -> 476,63 -> 478,63 -> 478,55 -> 478,63 -> 480,63 -> 480,56 -> 480,63 -> 482,63 -> 482,57 -> 482,63 -> 484,63 -> 484,55 -> 484,63 -> 486,63 -> 486,55 -> 486,63
497,147 -> 497,137 -> 497,147 -> 499,147 -> 499,146 -> 499,147 -> 501,147 -> 501,142 -> 501,147 -> 503,147 -> 503,144 -> 503,147 -> 505,147 -> 505,140 -> 505,147
494,118 -> 498,118
488,106 -> 488,97 -> 488,106 -> 490,106 -> 490,100 -> 490,106 -> 492,106 -> 492,101 -> 492,106 -> 494,106 -> 494,105 -> 494,106 -> 496,106 -> 496,96 -> 496,106 -> 498,106 -> 498,102 -> 498,106 -> 500,106 -> 500,101 -> 500,106 -> 502,106 -> 502,102 -> 502,106 -> 504,106 -> 504,100 -> 504,106
488,124 -> 488,126 -> 485,126 -> 485,129 -> 496,129 -> 496,126 -> 491,126 -> 491,124
488,106 -> 488,97 -> 488,106 -> 490,106 -> 490,100 -> 490,106 -> 492,106 -> 492,101 -> 492,106 -> 494,106 -> 494,105 -> 494,106 -> 496,106 -> 496,96 -> 496,106 -> 498,106 -> 498,102 -> 498,106 -> 500,106 -> 500,101 -> 500,106 -> 502,106 -> 502,102 -> 502,106 -> 504,106 -> 504,100 -> 504,106
488,106 -> 488,97 -> 488,106 -> 490,106 -> 490,100 -> 490,106 -> 492,106 -> 492,101 -> 492,106 -> 494,106 -> 494,105 -> 494,106 -> 496,106 -> 496,96 -> 496,106 -> 498,106 -> 498,102 -> 498,106 -> 500,106 -> 500,101 -> 500,106 -> 502,106 -> 502,102 -> 502,106 -> 504,106 -> 504,100 -> 504,106
497,147 -> 497,137 -> 497,147 -> 499,147 -> 499,146 -> 499,147 -> 501,147 -> 501,142 -> 501,147 -> 503,147 -> 503,144 -> 503,147 -> 505,147 -> 505,140 -> 505,147
488,106 -> 488,97 -> 488,106 -> 490,106 -> 490,100 -> 490,106 -> 492,106 -> 492,101 -> 492,106 -> 494,106 -> 494,105 -> 494,106 -> 496,106 -> 496,96 -> 496,106 -> 498,106 -> 498,102 -> 498,106 -> 500,106 -> 500,101 -> 500,106 -> 502,106 -> 502,102 -> 502,106 -> 504,106 -> 504,100 -> 504,106
488,124 -> 488,126 -> 485,126 -> 485,129 -> 496,129 -> 496,126 -> 491,126 -> 491,124
509,115 -> 513,115
497,147 -> 497,137 -> 497,147 -> 499,147 -> 499,146 -> 499,147 -> 501,147 -> 501,142 -> 501,147 -> 503,147 -> 503,144 -> 503,147 -> 505,147 -> 505,140 -> 505,147
488,106 -> 488,97 -> 488,106 -> 490,106 -> 490,100 -> 490,106 -> 492,106 -> 492,101 -> 492,106 -> 494,106 -> 494,105 -> 494,106 -> 496,106 -> 496,96 -> 496,106 -> 498,106 -> 498,102 -> 498,106 -> 500,106 -> 500,101 -> 500,106 -> 502,106 -> 502,102 -> 502,106 -> 504,106 -> 504,100 -> 504,106
506,173 -> 510,173
488,106 -> 488,97 -> 488,106 -> 490,106 -> 490,100 -> 490,106 -> 492,106 -> 492,101 -> 492,106 -> 494,106 -> 494,105 -> 494,106 -> 496,106 -> 496,96 -> 496,106 -> 498,106 -> 498,102 -> 498,106 -> 500,106 -> 500,101 -> 500,106 -> 502,106 -> 502,102 -> 502,106 -> 504,106 -> 504,100 -> 504,106
481,79 -> 481,81 -> 478,81 -> 478,89 -> 490,89 -> 490,81 -> 485,81 -> 485,79
465,76 -> 465,71 -> 465,76 -> 467,76 -> 467,72 -> 467,76 -> 469,76 -> 469,68 -> 469,76 -> 471,76 -> 471,73 -> 471,76 -> 473,76 -> 473,71 -> 473,76 -> 475,76 -> 475,70 -> 475,76 -> 477,76 -> 477,71 -> 477,76 -> 479,76 -> 479,73 -> 479,76 -> 481,76 -> 481,74 -> 481,76 -> 483,76 -> 483,72 -> 483,76
505,35 -> 510,35
465,76 -> 465,71 -> 465,76 -> 467,76 -> 467,72 -> 467,76 -> 469,76 -> 469,68 -> 469,76 -> 471,76 -> 471,73 -> 471,76 -> 473,76 -> 473,71 -> 473,76 -> 475,76 -> 475,70 -> 475,76 -> 477,76 -> 477,71 -> 477,76 -> 479,76 -> 479,73 -> 479,76 -> 481,76 -> 481,74 -> 481,76 -> 483,76 -> 483,72 -> 483,76
506,118 -> 510,118
465,76 -> 465,71 -> 465,76 -> 467,76 -> 467,72 -> 467,76 -> 469,76 -> 469,68 -> 469,76 -> 471,76 -> 471,73 -> 471,76 -> 473,76 -> 473,71 -> 473,76 -> 475,76 -> 475,70 -> 475,76 -> 477,76 -> 477,71 -> 477,76 -> 479,76 -> 479,73 -> 479,76 -> 481,76 -> 481,74 -> 481,76 -> 483,76 -> 483,72 -> 483,76
465,76 -> 465,71 -> 465,76 -> 467,76 -> 467,72 -> 467,76 -> 469,76 -> 469,68 -> 469,76 -> 471,76 -> 471,73 -> 471,76 -> 473,76 -> 473,71 -> 473,76 -> 475,76 -> 475,70 -> 475,76 -> 477,76 -> 477,71 -> 477,76 -> 479,76 -> 479,73 -> 479,76 -> 481,76 -> 481,74 -> 481,76 -> 483,76 -> 483,72 -> 483,76
503,150 -> 503,154 -> 501,154 -> 501,162 -> 513,162 -> 513,154 -> 507,154 -> 507,150
465,76 -> 465,71 -> 465,76 -> 467,76 -> 467,72 -> 467,76 -> 469,76 -> 469,68 -> 469,76 -> 471,76 -> 471,73 -> 471,76 -> 473,76 -> 473,71 -> 473,76 -> 475,76 -> 475,70 -> 475,76 -> 477,76 -> 477,71 -> 477,76 -> 479,76 -> 479,73 -> 479,76 -> 481,76 -> 481,74 -> 481,76 -> 483,76 -> 483,72 -> 483,76
509,171 -> 513,171
494,12 -> 494,13 -> 507,13 -> 507,12
509,38 -> 514,38
488,124 -> 488,126 -> 485,126 -> 485,129 -> 496,129 -> 496,126 -> 491,126 -> 491,124
465,76 -> 465,71 -> 465,76 -> 467,76 -> 467,72 -> 467,76 -> 469,76 -> 469,68 -> 469,76 -> 471,76 -> 471,73 -> 471,76 -> 473,76 -> 473,71 -> 473,76 -> 475,76 -> 475,70 -> 475,76 -> 477,76 -> 477,71 -> 477,76 -> 479,76 -> 479,73 -> 479,76 -> 481,76 -> 481,74 -> 481,76 -> 483,76 -> 483,72 -> 483,76
476,63 -> 476,58 -> 476,63 -> 478,63 -> 478,55 -> 478,63 -> 480,63 -> 480,56 -> 480,63 -> 482,63 -> 482,57 -> 482,63 -> 484,63 -> 484,55 -> 484,63 -> 486,63 -> 486,55 -> 486,63
495,44 -> 499,44
482,93 -> 496,93 -> 496,92
515,171 -> 519,171
465,76 -> 465,71 -> 465,76 -> 467,76 -> 467,72 -> 467,76 -> 469,76 -> 469,68 -> 469,76 -> 471,76 -> 471,73 -> 471,76 -> 473,76 -> 473,71 -> 473,76 -> 475,76 -> 475,70 -> 475,76 -> 477,76 -> 477,71 -> 477,76 -> 479,76 -> 479,73 -> 479,76 -> 481,76 -> 481,74 -> 481,76 -> 483,76 -> 483,72 -> 483,76
481,79 -> 481,81 -> 478,81 -> 478,89 -> 490,89 -> 490,81 -> 485,81 -> 485,79
522,177 -> 522,178 -> 536,178 -> 536,177
476,63 -> 476,58 -> 476,63 -> 478,63 -> 478,55 -> 478,63 -> 480,63 -> 480,56 -> 480,63 -> 482,63 -> 482,57 -> 482,63 -> 484,63 -> 484,55 -> 484,63 -> 486,63 -> 486,55 -> 486,63
497,147 -> 497,137 -> 497,147 -> 499,147 -> 499,146 -> 499,147 -> 501,147 -> 501,142 -> 501,147 -> 503,147 -> 503,144 -> 503,147 -> 505,147 -> 505,140 -> 505,147
515,167 -> 519,167
465,76 -> 465,71 -> 465,76 -> 467,76 -> 467,72 -> 467,76 -> 469,76 -> 469,68 -> 469,76 -> 471,76 -> 471,73 -> 471,76 -> 473,76 -> 473,71 -> 473,76 -> 475,76 -> 475,70 -> 475,76 -> 477,76 -> 477,71 -> 477,76 -> 479,76 -> 479,73 -> 479,76 -> 481,76 -> 481,74 -> 481,76 -> 483,76 -> 483,72 -> 483,76
506,112 -> 510,112
488,106 -> 488,97 -> 488,106 -> 490,106 -> 490,100 -> 490,106 -> 492,106 -> 492,101 -> 492,106 -> 494,106 -> 494,105 -> 494,106 -> 496,106 -> 496,96 -> 496,106 -> 498,106 -> 498,102 -> 498,106 -> 500,106 -> 500,101 -> 500,106 -> 502,106 -> 502,102 -> 502,106 -> 504,106 -> 504,100 -> 504,106
522,177 -> 522,178 -> 536,178 -> 536,177
465,76 -> 465,71 -> 465,76 -> 467,76 -> 467,72 -> 467,76 -> 469,76 -> 469,68 -> 469,76 -> 471,76 -> 471,73 -> 471,76 -> 473,76 -> 473,71 -> 473,76 -> 475,76 -> 475,70 -> 475,76 -> 477,76 -> 477,71 -> 477,76 -> 479,76 -> 479,73 -> 479,76 -> 481,76 -> 481,74 -> 481,76 -> 483,76 -> 483,72 -> 483,76
488,124 -> 488,126 -> 485,126 -> 485,129 -> 496,129 -> 496,126 -> 491,126 -> 491,124
487,133 -> 487,134 -> 501,134
497,121 -> 501,121
476,63 -> 476,58 -> 476,63 -> 478,63 -> 478,55 -> 478,63 -> 480,63 -> 480,56 -> 480,63 -> 482,63 -> 482,57 -> 482,63 -> 484,63 -> 484,55 -> 484,63 -> 486,63 -> 486,55 -> 486,63
488,106 -> 488,97 -> 488,106 -> 490,106 -> 490,100 -> 490,106 -> 492,106 -> 492,101 -> 492,106 -> 494,106 -> 494,105 -> 494,106 -> 496,106 -> 496,96 -> 496,106 -> 498,106 -> 498,102 -> 498,106 -> 500,106 -> 500,101 -> 500,106 -> 502,106 -> 502,102 -> 502,106 -> 504,106 -> 504,100 -> 504,106
488,106 -> 488,97 -> 488,106 -> 490,106 -> 490,100 -> 490,106 -> 492,106 -> 492,101 -> 492,106 -> 494,106 -> 494,105 -> 494,106 -> 496,106 -> 496,96 -> 496,106 -> 498,106 -> 498,102 -> 498,106 -> 500,106 -> 500,101 -> 500,106 -> 502,106 -> 502,102 -> 502,106 -> 504,106 -> 504,100 -> 504,106
489,50 -> 493,50
465,76 -> 465,71 -> 465,76 -> 467,76 -> 467,72 -> 467,76 -> 469,76 -> 469,68 -> 469,76 -> 471,76 -> 471,73 -> 471,76 -> 473,76 -> 473,71 -> 473,76 -> 475,76 -> 475,70 -> 475,76 -> 477,76 -> 477,71 -> 477,76 -> 479,76 -> 479,73 -> 479,76 -> 481,76 -> 481,74 -> 481,76 -> 483,76 -> 483,72 -> 483,76
512,169 -> 516,169
490,16 -> 490,20 -> 484,20 -> 484,26 -> 499,26 -> 499,20 -> 494,20 -> 494,16
494,32 -> 499,32
490,16 -> 490,20 -> 484,20 -> 484,26 -> 499,26 -> 499,20 -> 494,20 -> 494,16
488,106 -> 488,97 -> 488,106 -> 490,106 -> 490,100 -> 490,106 -> 492,106 -> 492,101 -> 492,106 -> 494,106 -> 494,105 -> 494,106 -> 496,106 -> 496,96 -> 496,106 -> 498,106 -> 498,102 -> 498,106 -> 500,106 -> 500,101 -> 500,106 -> 502,106 -> 502,102 -> 502,106 -> 504,106 -> 504,100 -> 504,106'''.split('\n')